In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
%matplotlib notebook

In [2]:
the_dtypes = {"date": str, "county": str, "state": str, "fips": object, "cases": int, "deaths": int}
the_na_vals = {"fips": "NaN"}

In [3]:
counties = pd.read_csv("us-counties.csv",
                       parse_dates=["date"],
                       infer_datetime_format=True,
                       dtype=the_dtypes,
                       na_values=the_na_vals).drop(axis=1, labels=["fips"])

In [4]:
ll = [counties[counties["county"] == 'New York City'],
      counties[(counties["county"] == "Fairfield") & (counties["state"] == "Connecticut")],
      counties[(counties["county"] == "New Haven") & (counties["state"] == "Connecticut")],
      counties[(counties["county"] == "Bergen") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Passaic") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Essex") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Union") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Middlesex") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Sussex") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Monmouth") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Morris") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Hunterdon") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Pike") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Putnam") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Somerset") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Mercer") & (counties["state"] == "New Jersey")],
      counties[(counties["county"] == "Westchester") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Rockland") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Putnam") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Orange") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Dutchess") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Nassau") & (counties["state"] == "New York")],
      counties[(counties["county"] == "Suffolk") & (counties["state"] == "New York")]
     ]

In [5]:
greater_nyc_all = pd.concat(ll)
greater_nyc = greater_nyc_all.pivot(index="date", columns="county", values="cases")
greater_nyc['NYC Metro Total Cases'] = greater_nyc.sum(axis=1)

In [6]:
us_ex_nyc_temp = counties.merge(greater_nyc_all, on=["state", "county"], how="left", indicator=True).query('_merge == "left_only"').drop(columns='_merge')
us_ex_nyc = us_ex_nyc_temp[us_ex_nyc_temp['cases_y'].isna()].drop(['date_y', 'cases_y', 'deaths_y'], axis=1)
us_ex_nyc = us_ex_nyc.rename(columns={'date_x': 'date', 'cases_x': 'cases', 'deaths_x': 'deaths'})
us_ex_nyc_all = us_ex_nyc.groupby(['date']).sum()
us_ex_nyc['location'] = us_ex_nyc['county'] + ' County, ' + us_ex_nyc['state']
us_ex_nyc = us_ex_nyc.pivot(index="date", columns="location", values="cases")
us_ex_nyc['Total US Cases ex NYC Metro'] = us_ex_nyc_all['cases']

In [10]:
greater_nyc['Total US Cases ex NYC Metro'] = us_ex_nyc_all['cases']

In [11]:
selection = greater_nyc.columns.to_list()

In [14]:
def select_county(county):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    greater_nyc['NYC Metro Total Cases'].plot(x="date", y="", label='NYC Metro Total Cases')
    greater_nyc[county].plot(x="date", y="cases", label=county)
    ax.set_xlabel("Date")
    ax.set_ylabel("Cases")
    ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:,.0f}"))
    ax.set_title("NYC Metro Total Cases vs {}".format(county))
    ax.legend(loc='upper left')

In [15]:
ipywidgets.interact(select_county, county=selection)

interactive(children=(Dropdown(description='county', options=('Bergen', 'Dutchess', 'Essex', 'Fairfield', 'Hun…

<function __main__.select_county(county)>

In [ ]:
us_ex_nyc_temp = counties.merge(greater_nyc_all, on=["state", "county"], how="left", indicator=True).query('_merge == "left_only"').drop(columns='_merge')
us_ex_nyc = us_ex_nyc_temp[us_ex_nyc_temp['cases_y'].isna()].drop(['date_y', 'cases_y', 'deaths_y'], axis=1)
us_ex_nyc = us_ex_nyc.rename(columns={'date_x': 'date', 'cases_x': 'cases', 'deaths_x': 'deaths'})
us_ex_nyc_all = us_ex_nyc.groupby(['date']).sum()
us_ex_nyc['location'] = us_ex_nyc['county'] + ' County, ' + us_ex_nyc['state']
us_ex_nyc = us_ex_nyc.pivot(index="date", columns="location", values="cases")

In [8]:
us_ex_nyc['Total US Cases ex NYC Metro'] = us_ex_nyc_all['cases']

In [ ]:
greater_nyc['Total US Cases'] = us_ex_nyc_all['cases']

In [ ]:
greater_nyc

In [ ]:
us_ex_nyc['location'] = us_ex_nyc['county'] + ' County, ' + us_ex_nyc['state'] 

In [ ]:
us_ex_nyc.pivot(index="date", columns="location", values="cases")

In [9]:
us_ex_nyc

location,"Abbeville County, South Carolina","Acadia County, Louisiana","Accomack County, Virginia","Ada County, Idaho","Adair County, Iowa","Adair County, Kentucky","Adair County, Missouri","Adair County, Oklahoma","Adams County, Colorado","Adams County, Idaho",...,"York County, Pennsylvania","York County, South Carolina","York County, Virginia","Young County, Texas","Yuba County, California","Yukon-Koyukuk Census Area County, Alaska","Yuma County, Arizona","Yuma County, Colorado","Zapata County, Texas",Total US Cases ex NYC Metro
date,,,,,,,,,,,,,,,,,,,,,
2020-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-11,9.0,98.0,15.0,517.0,1.0,11.0,11.0,27.0,593.0,1.0,...,293.0,127.0,27.0,3.0,15.0,1.0,15.0,2.0,3.0,302880
2020-04-12,9.0,99.0,15.0,517.0,1.0,43.0,11.0,27.0,647.0,1.0,...,307.0,134.0,29.0,3.0,15.0,1.0,17.0,2.0,3.0,319238
2020-04-13,9.0,101.0,15.0,529.0,1.0,43.0,11.0,27.0,693.0,1.0,...,331.0,136.0,33.0,3.0,15.0,1.0,17.0,2.0,3.0,335790
